In [ ]:
# 事前にGCPの認証が必要で、認証方法は環境によって異なる
# colabの場合:
#   セルで下記を実行
#     from google.colab import auth
#     auth.authenticate_user()
#     %env GCLOUD_PROJECT=GCPのプロジェクトID
# PC等のローカル環境の場合:
#   初回のみ、https://cloud.google.com/sdk/docs/install-sdk からgcloud CLIをインストールし、gcloud initを実行
%load_ext google.cloud.bigquery

# 5章 抽出
## 5-1 列名指定による列の選択


### Q: 集計対象の列のみに絞り込み


#### Not Awesome


In [ ]:
%%bigquery
-- 除外する列名を指定して選択
select
    * except (
        reserved_at,
        length_of_stay,
        total_price,
        people_num,
        status,
        canceled_at
    )
from example.reservation

#### Awesome


In [ ]:
%%bigquery
-- 列名を指定して列を選択
select
    reservation_id,
    hotel_id,
    customer_id,
    checkin_date,
    checkout_date
from example.reservation

## 5-3 条件指定による行の抽出


### Q: 宿泊人数が2〜4人の予約履歴のみ抽出


#### Not Awesome


In [ ]:
%%bigquery
select *
from example.reservation
where
    -- people_numが2以上、かつ4以下の行を抽出
    people_num >= 2
    and people_num <= 4

#### Awesome


In [ ]:
%%bigquery
select *
from example.reservation
-- people_numが2から4の範囲の行を抽出
where people_num between 2 and 4

## 5-4 ランダムサンプリング


### Q: 予約履歴をランダムサンプリング
#### Not Awesome 1


In [ ]:
%%bigquery
select *
from example.reservation
-- 最大20000件を抽出
limit 20000

#### Not Awesome 2


In [ ]:
%%bigquery
select *
from example.reservation
-- ランダムに並び替え
order by rand()
-- 最大20000件を抽出
limit 20000

#### Awesome


In [ ]:
%%bigquery
select *
from example.reservation
-- 0~1の乱数が0.01未満になる行のみ抽出
where rand() < 0.01

## 5-5 不均衡データの調整


### Q: 未キャンセルデータをキャンセル済データと同数になるようにアンダーサンプリング


#### Awesome


In [ ]:
%%bigquery
with
-- （1）未キャンセルデータを抽出
majority as (
    select *
    from example.reservation
    where status = "reserved"
),

-- （2）キャンセル済データを抽出
minority as (
    select *
    from example.reservation
    where status = "canceled"
)

--（3）未キャンセルデータをキャンセル済データと同数になるようにランダムサンプリング
select *
from majority
where
    rand() < (
        (select count(*) from minority) / (select count(*) from majority)
    )

--（4）ランダムサンプリングした未キャンセルデータをキャンセル済データと結合
union all
select *
from minority